In [ ]:
'''
Starter Code, solution in next cell
MLP character model. Code adapted from https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


def build_model(maxlen, chars):
    # build the model: an MLP with 1 hidden layer
    print('Build model...')
    model = Sequential()
    model.add(Flatten(input_shape=(maxlen, len(chars))))
    model.add(Dense(128))
    model.add(Dense(len(chars)))
    model.add(Activation('softmax'))

    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

if __name__ == '__main__':
    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

    model = build_model(maxlen, chars)

    import ipdb; 
    
    model.fit(
        x,
        y,
        batch_size=128,
        epochs=60,
        callbacks=[print_callback]
    )
    ipdb.set_trace()

In [3]:
'''
Solution:
Conv1D character model. Code adapted from https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
import numpy as np
import random
import sys
import io

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

# 2nd Dataset : Combined works of Shakespeare
# path = get_file('shakespeare_input.txt', origin='https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt')

with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

# print(x.shape[0])
# print(x.shape[1])
# print(x.shape[2])


def build_model(maxlen, chars):

    print('Build model...')
    model = Sequential()
    # model 1 : Simple Model, 1 Hidden layer
#     model.add(Conv1D(64, 5, padding='same', activation='relu',input_shape=(maxlen, len(chars))))
#     model.add(Flatten(input_shape = (maxlen, len(chars))))
    model.add(Conv1D(128, 5, padding='same', activation='relu', input_shape=(maxlen, len(chars))))
    model.add(Flatten())
    model.add(Dense(len(chars), activation='softmax'))  
        
    # model 2 : VGG net like architecture, 5 layers
#     model.add(Conv1D(64, 3, activation='relu',input_shape=(maxlen, len(chars))))
#     model.add(Conv1D(64, 3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Conv1D(128, 3, activation='relu'))
#     model.add(Conv1D(128, 3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Flatten())
#     model.add(Dense(len(chars), activation='softmax'))  
    
    #model 3: adapted from https://keras.io/getting-started/sequential-model-guide/
#     model.add(Conv1D(64, 3, activation='relu',input_shape=(maxlen, len(chars))))
#     model.add(Conv1D(64, 3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=3))
#     model.add(Conv1D(128, 3, activation='relu'))
#     model.add(Conv1D(128, 3, activation='relu'))
#     model.add(MaxPooling1D(pool_size=3))
#     model.add(GlobalAveragePooling1D())
#     model.add(Dropout(0.5))
# #     model.add(Flatten())
#     model.add(Dense(len(chars), activation='sigmoid'))
    
    
    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

if __name__ == '__main__':
    print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

    model = build_model(maxlen, chars)

    import ipdb; 
    
    model.fit(
        x,
        y,
        batch_size=128,
        epochs=60,
        callbacks=[print_callback]
    )
    ipdb.set_trace()

corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...
Build model...
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Epoch 1/60
200285/200285 [==============================] - 61s 307us/step - loss: 2.0227

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "himself: who has not already, in my situ"
himself: who has not already, in my situal complation, there are of the soment the
soment the
presuligion, there
the sament and propenisperians and diming there as the mere and there in the proposely in the stral finds, and but man there and presing there
such and the prestime the with the
propuland beation, but the
believery and and beappration, and man in prestime there the from the prestiming the proment, and and there is and and the
----- diversity: 0.5
----- Generating with seed: "himself: who has not already, in my situ"
himself: who has not already, in my situlate."
when there
disting prehere

.kpe consibe find a the somen a give all the south, world, and as
possible only as as in the somer,
the haves presirivicsolution and count, more alfsiness, of the dill
it of the such which the somedishys the samently and eurong respidies indentia
----- diversity: 1.0
----- Generating with seed: "e orbit, however independent of each oth"
e orbit, however independent of each othen."--as may cundom the parentdines to lood
tood--and and the our weate the degors playian, unally prom, as bolal. we cogy to appeare, littem one gen the ex  he e n le a the ption
tave; arder of the raporythink humanitagres babacirutire to subness,
rarbul
of religious bidity it the hise the of cognd herdidid as inted, the too are not this opertaing
of be upon, in colardiratule, only secisily are n
----- diversity: 1.2
----- Generating with seed: "e orbit, however independent of each oth"
e orbit, however independent of each otherse, coudify-pre. your agres it greancys. and with a"retmainy as tavidtitation, "sers, 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:98: RuntimeWarning: divide by zero encountered in log


s ours
fil the man inde his wantition, the
imake thend,
"breave, has seemink
decyation this devation in fech somek one to objects sawellness,
it liv
Epoch 5/60
200285/200285 [==============================] - 66s 331us/step - loss: 1.8669

----- Generating text after Epoch: 4
----- diversity: 0.2
----- Generating with seed: "nt
(over chosen disciples or members of "
nt
(over chosen disciples or members of somen the of and indemply and and and and the which the the more as in the to the considence and distrustain the and the which the which the struloted of the to subd the soulsti--assion, the considence in of all of the destruth, the
consupeor and and and and and as of isself, the same and and of the is the the prosulty and and the many
the with,
which itself and of in and the and somethings, the p
----- diversity: 0.5
----- Generating with seed: "nt
(over chosen disciples or members of "
nt
(over chosen disciples or members of the longe, are
consike confined and not the diste the the 

actual calnenurfulpinbasure, milition-oags not on itter; thong, "go, always be ence the as utt in in arigin live larguequisfess sy fright good sxeisy fasse
at and as must rird himi, it
dogd bert ourgirings motiably most ourstlied to to ethive found in and it the docpn the caims grasclused
cown and be uved by a caseven appinity our of ganeeration, that sament pople
ingaboficile scarnorposseite self also
----- diversity: 1.2
----- Generating with seed: "living according to nature," means
actua"
living according to nature," means
actuanivert cruld ptways a so o hre ex  we a cee in near weikg chalm
lalocreasinces remardures to inchic, upetans of another. and pusm! ipmes intedness pariacs, bocusandarroration this of the worary a kin isway "de an of free the fre hit the necery inse sibaughly
memk on suct--viruely most this now have tetterhing, respectionongaccipeks reuing uspansy time an what nation cate thilatical cing, prasy dei
Epoch 9/60
200285/200285 [==============================] - 5

KeyboardInterrupt: 

In [ ]:
'''
Using LSTMs, ;)

Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])